# TransversalHadamard Fix - Comprehensive Test

This notebook tests the `FaultTolerantGadgetExperiment` with `TransversalHadamard` across multiple quantum error correction codes to verify the fix works correctly.

## 1. Setup Environment and Clear Module Cache

In [ ]:
import sys

# Clear any cached qectostim modules to ensure fresh imports
modules_to_remove = [key for key in list(sys.modules.keys()) if 'qectostim' in key]
for mod in modules_to_remove:
    del sys.modules[mod]
print(f"Cleared {len(modules_to_remove)} cached qectostim modules")

# Add source path
src_path = "/Users/scottjones_admin/Library/Mobile Documents/com~apple~CloudDocs/Mac files/Repos/QECToStim/src"
if src_path not in sys.path:
    sys.path.insert(0, src_path)
print(f"Added source path: {src_path}")

## 2. Import Required Libraries and Codes

In [ ]:
# Import codes
from qectostim.codes.small import SteanCode713, FourQubit422Code, PerfectCode513
from qectostim.codes.small.non_css_codes import BareAncillaCode713

# Import gadget
from qectostim.gadgets.transversal import TransversalHadamard

# Import experiment
from qectostim.experiments.ft_gadget_experiment import FaultTolerantGadgetExperiment

# Import noise model
from qectostim.noise.circuit_noise import CircuitDepolarizingNoise

print("✓ All imports successful!")

## 3. Define Test Function for TransversalHadamard

In [ ]:
import traceback

def test_transversal_hadamard(code_class, code_name):
    """Test TransversalHadamard with a given code class."""
    result = {
        "code": code_name,
        "status": "FAILED",
        "n": None,
        "k": None,
        "d": None,
        "detectors": None,
        "error_mechanisms": None,
        "error": None
    }
    
    try:
        # Create code instance
        code = code_class()
        result["n"] = code.n
        result["k"] = code.k
        result["d"] = code.d
        print(f"  ✓ Created code instance: [{code.n},{code.k},{code.d}]")
        
        # Create gadget
        gadget = TransversalHadamard()
        print(f"  ✓ Created TransversalHadamard gadget")
        
        # Create noise model
        noise = CircuitDepolarizingNoise(p1=1e-3, p2=1e-3)
        print(f"  ✓ Created CircuitDepolarizingNoise(p1=1e-3, p2=1e-3)")
        
        # Create experiment
        exp = FaultTolerantGadgetExperiment(
            codes=[code],
            gadget=gadget,
            noise_model=noise,
            num_rounds_before=2,
            num_rounds_after=2
        )
        print(f"  ✓ Created FaultTolerantGadgetExperiment (rounds=2/2)")
        
        # Convert to stim circuit
        circuit = exp.to_stim()
        print(f"  ✓ Generated Stim circuit ({circuit.num_qubits} qubits, {circuit.num_detectors} detectors)")
        
        # Generate detector error model (non-deterministic errors would fail here)
        dem = circuit.detector_error_model()
        
        # Count error mechanisms
        num_errors = len([inst for inst in dem.flattened() if inst.type == "error"])
        
        result["detectors"] = circuit.num_detectors
        result["error_mechanisms"] = num_errors
        result["status"] = "PASSED"
        print(f"  ✓ Generated Detector Error Model ({num_errors} error mechanisms)")
        print(f"\n  ★ PASSED ★")
        
    except Exception as e:
        result["error"] = str(e)
        print(f"\n  ✗ FAILED: {e}")
        print(f"\n  Traceback:\n{traceback.format_exc()}")
    
    return result

# Store all results
all_results = []
print("Test function defined successfully!")

## 4. Test SteanCode713

In [ ]:
print("=" * 60)
print("Testing: SteanCode713")
print("=" * 60)
result = test_transversal_hadamard(SteanCode713, "SteanCode713")
all_results.append(result)

## 5. Test FourQubit422Code

In [ ]:
print("=" * 60)
print("Testing: FourQubit422Code")
print("=" * 60)
result = test_transversal_hadamard(FourQubit422Code, "FourQubit422Code")
all_results.append(result)

## 6. Test BareAncillaCode713

In [ ]:
print("=" * 60)
print("Testing: BareAncillaCode713")
print("=" * 60)
result = test_transversal_hadamard(BareAncillaCode713, "BareAncillaCode713")
all_results.append(result)

## 7. Test PerfectCode513

In [ ]:
print("=" * 60)
print("Testing: PerfectCode513")
print("=" * 60)
result = test_transversal_hadamard(PerfectCode513, "PerfectCode513")
all_results.append(result)

## 8. Summary of Test Results

In [ ]:
print("\n" + "=" * 70)
print("COMPREHENSIVE TEST SUMMARY")
print("=" * 70)

passed = [r for r in all_results if r["status"] == "PASSED"]
failed = [r for r in all_results if r["status"] == "FAILED"]

print(f"\nTotal codes tested: {len(all_results)}")
print(f"Passed: {len(passed)}")
print(f"Failed: {len(failed)}")

if passed:
    print("\n✓ PASSED CODES:")
    print("-" * 70)
    print(f"{'Code':<25} {'[n,k,d]':<12} {'Detectors':<12} {'Errors':<12}")
    print("-" * 70)
    for r in passed:
        nkd = f"[{r['n']},{r['k']},{r['d']}]"
        print(f"{r['code']:<25} {nkd:<12} {r['detectors']:<12} {r['error_mechanisms']:<12}")

if failed:
    print("\n✗ FAILED CODES:")
    print("-" * 70)
    for r in failed:
        print(f"  {r['code']}: {r['error']}")

print("\n" + "=" * 70)
if len(passed) == len(all_results):
    print("🎉 ALL TESTS PASSED!")
    print("TransversalHadamard fix is working correctly across all codes!")
    print("No non-deterministic detector errors were found.")
else:
    print(f"⚠️  {len(failed)} test(s) failed - review errors above")
print("=" * 70)